<a href="https://colab.research.google.com/github/ztjfreedom/colab/blob/master/pytorch_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [0]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

In [0]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

![CNN](https://img-blog.csdn.net/20180211155818513?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvbTBfMzczMDYzNjA=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input channel, 10 output channels, kernel 5*5
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        # fully connect
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        # in_size = 64
        in_size = x.size(0) # one batch
        # x: 64*10*12*12
        x = F.relu(self.mp(self.conv1(x)))
        # x: 64*20*4*4
        x = F.relu(self.mp(self.conv2(x)))
        # x: 64*320
        x = x.view(in_size, -1) # flatten the tensor
        # x: 64*10
        x = self.fc(x)
        return F.log_softmax(x)

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Net().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

print(device)

cuda


In [0]:
def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def test():
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True).to(device), Variable(target).to(device)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [25]:
for epoch in range(1, 10):
    train(epoch)
    test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.317667
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.504209
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.501078
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.187059
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.271830


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1881, Accuracy: 9435/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.089617
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.096332
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.248474
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.036451
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.142975

Test set: Average loss: 0.1123, Accuracy: 9663/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.176653
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.041410
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.128798
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.060153
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.080191

Test set: Average loss: 0.0874, Accuracy: 9732/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.064529
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.114987
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.048670
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.099151
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.253495

Test set: Average loss: 0.0720, Accuracy: 9777/10000 (97%)

Train Epoch: 5 [0/600